# Data Collection
### traffic data, weather data, events data(Public holidays)

In [2]:
import pandas as pd
import numpy as np

# Load the traffic data
traffic_data = pd.read_csv('Dataset_Uber Traffic.csv')
traffic_data['DateTime'] = pd.to_datetime(traffic_data['DateTime'])

# Load the weather data
weather_data = pd.read_csv('weather.csv')
weather_data['DateTime'] = pd.to_datetime(weather_data['date_time'])

# Load the event data
event_data = pd.read_csv('Holidays2018.csv')
event_data['DateTime'] = pd.to_datetime(event_data['date'])  # Convert 'date' to 'DateTime'

C:\Users\sayali\AppData\Local\Temp\ipykernel_16736\79064363.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_data['DateTime'] = pd.to_datetime(traffic_data['DateTime'])


# data synchronizing based on timestamps to align traffic data with corresponding weather conditions and events.

In [3]:
# Extract the year from the traffic data to align with
traffic_year = traffic_data['DateTime'].dt.year.unique()[0]

def adjust_year(df, target_year):
    def replace_year(x):
        try:
            return x.replace(year=target_year)
        except ValueError:
            # Handle the February 29 case for leap years
            if x.month == 2 and x.day == 29:
                return x.replace(month=2, day=28, year=target_year)
            else:
                raise

    df['DateTime'] = df['DateTime'].apply(replace_year)
    return df

# Adjust the year in weather data
weather_data = adjust_year(weather_data, traffic_year)

# Adjust the year in event data
event_data = adjust_year(event_data, traffic_year)

# Data Integration

### Merge the data

In [4]:
# Merge the datasets on DateTime
combined_data = pd.merge(traffic_data, weather_data, on='DateTime', how='left')
combined_data = pd.merge(combined_data, event_data, on='DateTime', how='left')

### cleaning

In [5]:
# Remove duplicates
combined_data = combined_data.drop_duplicates()

# Handle missing values
combined_data = combined_data.fillna(method='ffill')  # Example: forward fill

C:\Users\sayali\AppData\Local\Temp\ipykernel_16736\547528690.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data = combined_data.fillna(method='ffill')  # Example: forward fill


### Normalize

In [6]:
print(combined_data.columns)

Index(['DateTime', 'Junction', 'Vehicles', 'ID', 'date_time', 'maxtempC',
       'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex', 'uvIndex.1',
       'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset',
       'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph',
       'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
       'winddirDegree', 'windspeedKmph', 'date', 'day', 'holiday',
       'holiday_type'],
      dtype='object')


In [7]:
from sklearn.preprocessing import StandardScaler

# Normalize the relevant columns
scaler = StandardScaler()
combined_data[['tempC', 'humidity', 'windspeedKmph']] = scaler.fit_transform(combined_data[['tempC', 'humidity', 'windspeedKmph']])

# Save the cleaned and merged data
combined_data.to_csv('combined_data.csv', index=False)

### Displaying merged data

In [8]:
# Displaying the first 10 rows
print(combined_data.head(10))

    DateTime  Junction  Vehicles           ID            date_time  maxtempC  \
0 2015-01-11         1        15  20151101001  2009-01-11 00:00:00      27.0   
1 2015-01-11         1        15  20151101001  2010-01-11 00:00:00      26.0   
2 2015-01-11         1        15  20151101001  2011-01-11 00:00:00      28.0   
3 2015-01-11         1        15  20151101001  2012-01-11 00:00:00      29.0   
4 2015-01-11         1        15  20151101001  2013-01-11 00:00:00      29.0   
5 2015-01-11         1        15  20151101001  2014-01-11 00:00:00      28.0   
6 2015-01-11         1        15  20151101001  2015-01-11 00:00:00      26.0   
7 2015-01-11         1        15  20151101001  2016-01-11 00:00:00      27.0   
8 2015-01-11         1        15  20151101001  2017-01-11 00:00:00      26.0   
9 2015-01-11         1        15  20151101001  2018-01-11 00:00:00      27.0   

   mintempC  totalSnow_cm  sunHour  uvIndex  ...  precipMM  pressure  \
0      15.0           0.0     11.6      6.0  ..